In [6]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# load data

In [2]:
data = np.load('simulation_data.npy')
latent = np.load('simulation_latent.npy')

In [3]:
data.shape, latent.shape

((10, 20, 20), (10, 20))

# parameters

In [4]:
l_penalty = 0.02
step_size = 0.005
iter_max = 50000

# proximal operator

In [14]:
def prox_l2(beta, s, l):
    n = nd.array(beta).shape[0]
    
    banded = np.block([
        [np.zeros([1,1]), (-1)*s*l*np.ones([1,n-1])],
        [(1+s*l)*np.ones([1,1]), (1+2*s*l)*np.ones([1,n-2]), (1+s*l)*np.ones([1,1])],
        [(-1)*s*l*np.ones([1,n-1]), np.zeros([1,1])]
    ])
    
    return sp.linalg.solve_banded((1,1), banded, beta, True, True, False)

# gradient operator

In [15]:
def gradient(beta, l):
    pass